In [1]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
import dwave.inspector

In [2]:
# select quadratic program to solve
qp = QuadraticProgram()
qp.read_from_lp_file("example.lp")
qubo = QuadraticProgramToQubo().convert(qp)
bqm_binary = dimod.as_bqm(qubo.objective.linear.to_array(), qubo.objective.quadratic.to_array(), dimod.BINARY)
print(qp)
print(bqm_binary)
bqm_ising = bqm_binary.to_ising()
print("Ising Model form coefficients:")
print(bqm_ising)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: example_qp

Minimize
 obj: - x_0 - x_2 + [ - 2 x_0^2 + 2 x_0*x_2 - 4 x_2*x_1 + 2 x_1^2 ]/2
Subject To
 A0_eq_b0: - x_2 + x_1 = 0

Bounds
 0 <= x_0 <= 1
 0 <= x_2 <= 1
 0 <= x_1 <= 1

Binaries
 x_0 x_2 x_1
End

AdjVectorBQM({0: -2.0, 1: 7.0, 2: 9.0}, {(0, 1): 1.0, (1, 2): -18.0}, 0.0, 'BINARY')
Ising Model form coefficients:
({0: -0.75, 1: -0.75, 2: 0.0}, {(0, 1): 0.25, (1, 2): -4.5}, 2.75)


## Solve on advantage system

In [3]:
sampler = EmbeddingComposite(DWaveSampler())
res = sampler.sample(bqm_binary, label="example_qubo_3_1024", num_reads=1024)
res.to_pandas_dataframe()

,0,1,2,chain_break_fraction,energy,num_occurrences
0,1,1,1,0.0,-3.0,629
1,1,0,0,0.0,-2.0,162
2,0,1,1,0.0,-2.0,216
3,0,0,0,0.0,0.0,17


In [4]:
dwave.inspector.show(res) # this is a good looking one

'http://127.0.0.1:18000/?problemId=e084281c-1653-45c1-834c-fe3157c8fad7'